# NL2SQL Pipeline Demo - Azure AI Agent Service

## Overview
This notebook demonstrates the **Azure AI Agent Service** implementation of our Natural Language to SQL pipeline.

### Key Features
- 🤖 **Persistent AI Agents** - Agents remain in Azure AI Foundry (not deleted after use)
- 🔐 **Enterprise Authentication** - Uses DefaultAzureCredential (Azure CLI, Managed Identity)
- 📊 **Built-in Observability** - Native Azure AI Foundry tracing
- ⚡ **Performance** - Agent reuse eliminates creation overhead
- 🎯 **2-Agent Architecture** - Intent extraction + SQL generation

### Pipeline Flow
```
Natural Language Query
         ↓
    [Intent Agent] ← Persistent agent in Azure AI Foundry
         ↓
    Intent JSON (entities, filters, metrics)
         ↓
    [SQL Agent] ← Persistent agent with schema context
         ↓
    Generated T-SQL
         ↓
    SQL Execution (Azure SQL)
         ↓
    Formatted Results + Token Usage + Cost
```

### Comparison with LangChain
| Feature | LangChain | Azure AI Agent Service |
|---------|-----------|------------------------|
| Orchestration | Prompt chains | Agents + Threads |
| Authentication | API Key | DefaultAzureCredential |
| State | Stateless | Stateful threads |
| Persistence | No | Yes (Azure AI Foundry) |
| Dependencies | 3 packages | 2 packages |

---

**Let's explore the pipeline step by step!**

## Step 1: Import Required Libraries

We'll import the Azure AI Agent Service SDK and other necessary modules.

In [ ]:
import os
import sys
import json
import time
from datetime import datetime
from dotenv import load_dotenv

# Azure AI Agent Service
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential

# Add current directory to path for local imports
sys.path.insert(0, os.path.dirname(os.path.abspath('__file__')))

print("✓ All libraries imported successfully!")
print(f"  - azure-ai-projects: Available")
print(f"  - azure-identity: Available")
print(f"  - Python version: {sys.version.split()[0]}")

## Step 2: Load Environment Configuration

Load Azure AI Foundry project configuration and verify settings.

In [ ]:
# Load environment variables
load_dotenv()

# Azure AI Foundry configuration
PROJECT_ENDPOINT = os.getenv("PROJECT_ENDPOINT")
MODEL_DEPLOYMENT_NAME = os.getenv("MODEL_DEPLOYMENT_NAME")

# Azure SQL configuration
AZURE_SQL_SERVER = os.getenv("AZURE_SQL_SERVER")
AZURE_SQL_DB = os.getenv("AZURE_SQL_DB")

print("Environment Configuration:")
print("=" * 60)
print(f"✓ Azure AI Project: {PROJECT_ENDPOINT}")
print(f"✓ Model Deployment: {MODEL_DEPLOYMENT_NAME}")
print(f"✓ Azure SQL Server: {AZURE_SQL_SERVER}")
print(f"✓ Database: {AZURE_SQL_DB}")
print("=" * 60)

# Verify required variables
if not PROJECT_ENDPOINT or not MODEL_DEPLOYMENT_NAME:
    raise ValueError("Missing required environment variables. Check .env file!")

## Step 3: Initialize Azure AI Project Client

Connect to Azure AI Foundry using DefaultAzureCredential (Azure CLI authentication).

In [ ]:
# Initialize Azure AI Project Client
project_client = AIProjectClient(
    endpoint=PROJECT_ENDPOINT,
    credential=DefaultAzureCredential()
)

print("✓ Azure AI Project Client initialized successfully!")
print(f"  Endpoint: {PROJECT_ENDPOINT}")
print(f"  Authentication: DefaultAzureCredential (Azure CLI)")
print("\n📌 Note: This uses your Azure CLI login (az login)")
print("   No API keys stored in code!")

## Step 4: Load Database Schema

Load the CONTOSO-FI database schema that will be provided to the SQL generation agent.

In [ ]:
from schema_reader import get_sql_database_schema_context

# Load database schema
schema_context = get_sql_database_schema_context()

print("✓ Database schema loaded successfully!")
print(f"  Schema size: {len(schema_context)} characters")
print(f"  Database: {AZURE_SQL_DB}")
print("\n📋 Schema Preview (first 300 chars):")
print("=" * 60)
print(schema_context[:300] + "...")
print("=" * 60)

## Step 5: Create Persistent Agents

Now we'll create our two persistent agents that will remain in Azure AI Foundry.

### Agent 1: Intent Extractor
Analyzes natural language queries to extract:
- Intent (count, list, aggregate, filter)
- Entities (tables, columns)
- Metrics to calculate
- Filters and conditions
- Grouping requirements

### Agent 2: SQL Generator
Generates T-SQL queries using:
- Intent from Agent 1
- Database schema context
- Azure SQL best practices

In [ ]:
# Create Intent Extraction Agent
intent_agent = project_client.agents.create_agent(
    model=MODEL_DEPLOYMENT_NAME,
    name="intent-extractor-demo",
    instructions="""You are an AI assistant that extracts the intent and entities from natural language database queries.

Analyze the user's question and provide:
1. The main intent (e.g., count, list, aggregate, filter)
2. Key entities mentioned (tables, columns, metrics)
3. Any filters or conditions
4. Desired aggregations or groupings

Return your analysis in JSON format with keys: intent, entity, metrics, filters, group_by."""
)

print("✓ Intent Extraction Agent created!")
print(f"  Agent ID: {intent_agent.id}")
print(f"  Model: {MODEL_DEPLOYMENT_NAME}")
print(f"  Name: {intent_agent.name}")
print()

# Create SQL Generation Agent
sql_agent = project_client.agents.create_agent(
    model=MODEL_DEPLOYMENT_NAME,
    name="sql-generator-demo",
    instructions=f"""You are an expert SQL query generator for Azure SQL Database.

Given the user's intent and the database schema, generate a valid T-SQL query.

Database Schema:
{schema_context}

Requirements:
- Generate clean, efficient T-SQL
- Use proper JOINs when needed
- Include appropriate WHERE clauses for filters
- Use meaningful column aliases
- Return ONLY the SQL query, no explanations
- Do NOT include markdown code blocks"""
)

print("✓ SQL Generation Agent created!")
print(f"  Agent ID: {sql_agent.id}")
print(f"  Model: {MODEL_DEPLOYMENT_NAME}")
print(f"  Name: {sql_agent.name}")
print(f"  Schema size: {len(schema_context)} characters")
print()
print("=" * 60)
print("🎉 Both agents are now persistent in Azure AI Foundry!")
print("   They will remain even after this notebook session ends.")
print("=" * 60)

## Step 6: Test Intent Extraction

Let's test the Intent Extraction agent with a sample query.

In [ ]:
# Test query
test_query = "How many customers do we have?"

print(f"🔍 Test Query: '{test_query}'")
print("\n" + "=" * 60)

# Create a thread for this conversation
thread = project_client.agents.threads.create()

# Send the query to the intent agent
project_client.agents.messages.create(
    thread_id=thread.id,
    role="user",
    content=test_query
)

# Run the agent
run = project_client.agents.runs.create_and_process(
    thread_id=thread.id,
    agent_id=intent_agent.id
)

# Get the response
messages = project_client.agents.messages.list(thread_id=thread.id)
for message in messages:
    if message.role == "assistant":
        intent_result = message.text_messages[0].text.value
        break

print("✓ Intent Extraction Result:")
print("=" * 60)
print(intent_result)
print("=" * 60)

# Parse JSON for better display
try:
    intent_json = json.loads(intent_result)
    print("\n📊 Structured Intent:")
    for key, value in intent_json.items():
        print(f"  {key}: {value}")
except:
    pass

## Step 7: Test SQL Generation

Now let's use the SQL agent to generate a query based on the extracted intent.

In [ ]:
print(f"📝 Generating SQL for intent: {intent_result}")
print("\n" + "=" * 60)

# Create a new thread for SQL generation
sql_thread = project_client.agents.threads.create()

# Send the intent to the SQL agent
project_client.agents.messages.create(
    thread_id=sql_thread.id,
    role="user",
    content=f"Intent: {intent_result}\n\nGenerate the SQL query:"
)

# Run the SQL agent
sql_run = project_client.agents.runs.create_and_process(
    thread_id=sql_thread.id,
    agent_id=sql_agent.id
)

# Get the SQL response
sql_messages = project_client.agents.messages.list(thread_id=sql_thread.id)
for message in sql_messages:
    if message.role == "assistant":
        generated_sql = message.text_messages[0].text.value
        break

print("✓ Generated SQL Query:")
print("=" * 60)
print(generated_sql)
print("=" * 60)